In [0]:
!unzip '/content/drive/My Drive/Steering_Datasets_Rev1 (1)/center_only.zip'

## Dataloader for training data 

In [0]:
import torch 
from torchvision import transforms, utils 
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd 

class SteeringDataset(Dataset):
    def __init__(self, image_path, data_transforms=None):
        """
        Your code here
        Hint: Use the python csv library to parse labels.csv
        """
        # self.name_label_path = name_label_path
        self.image_path = image_path
        self.name = pd.read_csv(self.image_path + 'interpolated.csv', usecols=range(5,6))

        self.labels = pd.read_csv(self.image_path + 'interpolated.csv', usecols=range(6,7))
        self.center_data = pd.concat([self.name, self.labels], axis=1) #combine image name and label dataframes
        self.center_data = self.center_data[self.center_data["filename"].str.contains('center')] # only keep center image names and labels

        self.name = pd.DataFrame(self.center_data[self.center_data.columns[0]]) # center images names
        self.len = self.name.shape[0]
        self.labels = pd.DataFrame(self.center_data[self.center_data.columns[1]]) # center image labels

        # bin angles into 100 classes

        # classes = 100
        # bins = np.linspace(-2.1,2,classes)
        # labels = np.linspace(1,classes-1,classes-1).astype(int)
        # self.labels = pd.cut(self.labels['angle'], bins = bins, labels = labels)

        # print(self.name)

    def __len__(self):
        """
        Your code here
        """
        return self.len

    def __getitem__(self, idx):
        """
        Your code here
        return a tuple: img, label
        """
        # img = Image.open(str(self.name.iloc[idx, 0]))
        img = Image.open(self.image_path + str(self.name.iloc[idx, 0])[7:]) # [7:] removes 'center/' before the image number since the zipped images just have numbers
        img = img.crop((0, 240, 640, 480))  # crop image (remove above horizon). Original dimensions: 640,480. changing to 640,240    
        img = img.resize((224,224)) #resize image --> pretrained alexnet model needs img sizes of 224 x 224
        transform = transforms.ToTensor()
        img = transform(img)
        label = self.labels.iloc[idx][0]
        return (img, label)

## Dataloader for test set 

In [0]:
import torch 
import torchvision 
from torch.utils.data import Dataset, DataLoader 
from torchvision import transforms 

from PIL import Image 
import numpy as np 
import os 
import pandas as pd 

class TestSet(Dataset): 
  def __init__(self, image_path, data_transform): 
    self.image_path = image_path 
    self.data_transform = data_transform 

    self.dataset = pd.read_csv('/content/drive/My Drive/Steering_Datasets_Rev1 (1)/CH2_002_Test/steering.csv') 
    print(type(self.dataset.iloc[0,0]))
  def __len__(self): 
    return len(self.dataset) 

  def __getitem__(self, idx): 
    img_name = os.path.join(self.image_path, str(self.dataset.iloc[idx,0])+'.jpg')
    img = Image.open(img_name)
    img = img.crop((0,240,640,480))
    img = img.resize((224,224))
    if self.data_transform: 
      img = self.data_transform(img)

    label = self.dataset.iloc[idx,1] 

    item = (img,label)
    return item 


## test to see if this works 
img_path = '/content/drive/My Drive/Steering_Datasets_Rev1 (1)/CH2_002_Test/center/'
test_data = TestSet(image_path=img_path, data_transform=transforms.ToTensor()) 
print(len(test_data)) 
# for idx, (data, label) in enumerate(test_data): 
#   print(f'idx: {idx},   data: {data.shape},   label:{label}')

<class 'numpy.int64'>
5614


## Loss function 

In [0]:
import torch.nn as nn 

class ClassificationLoss(torch.nn.Module): 
  def forward(self,input,target):
    m = nn.MSELoss() 
    return torch.sqrt(m(input.view(input.size(0)),target))

## Train loader and Model 

In [0]:
################################# LOADER ######################################################################
train_set = SteeringDataset(image_path='/content/center_only/', data_transforms=transforms.ToTensor())
for idx, (data, label) in enumerate(train_set): 
  print(f'Image size: {data.shape}') 
  if idx == 0: 
    break  

B = 16
train_loader = DataLoader(dataset=train_set, batch_size=B, shuffle=True)

#################################### MODEL #####################################################################
import torchvision.models as models 

alexnet = models.alexnet(pretrained=False, progress=True)
for param in alexnet.parameters(): 
  param.requires_grad = True 
alexnet.classifier[6] = nn.Linear(4096,1)


Image size: torch.Size([3, 224, 224])


## Train on Data 

In [0]:
class Args(): 
  def __init__(self): 
    self.num_epochs = 2 
    self.learning_rate = 0.001 
    self.log_interval = 20 

args = Args() 

import torch.optim as optim 
def train(args, model, train_loader): 
  model.train() 

  # define optimizer and loss 
  optimizer = optim.Adam(model.parameters(), lr=args.learning_rate) 
  criterion = ClassificationLoss() 

  train_losses = [] 
  loss_list = [] 
  for epoch in range(1,args.num_epochs+1):
    ### 
    for batch_idx, (data, target) in enumerate(train_loader): 
      optimizer.zero_grad() 
      output = model(data)
      loss = criterion(output,target)
      loss.backward() 
      optimizer.step() 

      loss_list.append(loss.item()) 
      if batch_idx % args.log_interval == 0: 
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, (batch_idx+1) * len(data), len(train_loader)*len(data),
                  100. * batch_idx / len(train_loader), loss.item()))
        
        train_losses.append(loss.item())
        torch.save(model.state_dict(), './pretrained_alexnet.pt')
    print(f'\n Average loss for Epoch {epoch}: {sum(loss_list)/len(loss_list)} \n')
    del loss_list[:]


train(args, alexnet, train_loader) 

Train Epoch: 1 [16/33808 (0%)]	Loss: 0.280956
Train Epoch: 1 [336/33808 (1%)]	Loss: 0.346788
Train Epoch: 1 [656/33808 (2%)]	Loss: 0.360874
Train Epoch: 1 [976/33808 (3%)]	Loss: 0.213135
Train Epoch: 1 [1296/33808 (4%)]	Loss: 0.462182
Train Epoch: 1 [1616/33808 (5%)]	Loss: 0.176479
Train Epoch: 1 [1936/33808 (6%)]	Loss: 0.179577
Train Epoch: 1 [2256/33808 (7%)]	Loss: 0.283545
Train Epoch: 1 [2576/33808 (8%)]	Loss: 0.356300
Train Epoch: 1 [2896/33808 (9%)]	Loss: 0.233821
Train Epoch: 1 [3216/33808 (9%)]	Loss: 0.388968
Train Epoch: 1 [3536/33808 (10%)]	Loss: 0.321891
Train Epoch: 1 [3856/33808 (11%)]	Loss: 0.139951
Train Epoch: 1 [4176/33808 (12%)]	Loss: 0.302057
Train Epoch: 1 [4496/33808 (13%)]	Loss: 0.232436
Train Epoch: 1 [4816/33808 (14%)]	Loss: 0.276607
Train Epoch: 1 [5136/33808 (15%)]	Loss: 0.164164
Train Epoch: 1 [5456/33808 (16%)]	Loss: 0.298527
Train Epoch: 1 [5776/33808 (17%)]	Loss: 0.180726
Train Epoch: 1 [6096/33808 (18%)]	Loss: 0.355537
Train Epoch: 1 [6416/33808 (19%)]	Lo

## Test

In [0]:

# create dataloader 
img_path = '/content/drive/My Drive/Steering_Datasets_Rev1 (1)/CH2_002_Test/center/'
test_data = TestSet(image_path=img_path, data_transform=transforms.ToTensor()) 
test_loader = DataLoader(dataset=test_data, batch_size=14, shuffle=True)

def test(model, data_loader): 
  model.eval() 

  # define loss 
  criterion = ClassificationLoss() 
  
  test_loss = 0 
  with torch.no_grad(): 
    for batch_idx, (data, target) in enumerate(data_loader): 
      output = model(data)
      loss = criterion(output,target)
      test_loss += loss.item() 

      print(f'Batch_idx: {batch_idx}   Loss: {loss}')

    total_loss = test_loss/len(data_loader) 
    print(f'Total Loss: {total_loss}')
    return total_loss 



test(alexnet, test_loader)

<class 'numpy.int64'>
Batch_idx: 0   Loss: 0.23220106959342957
Batch_idx: 1   Loss: 0.1281196027994156
Batch_idx: 2   Loss: 0.1493135392665863
Batch_idx: 3   Loss: 0.31417790055274963
Batch_idx: 4   Loss: 0.2530134618282318
Batch_idx: 5   Loss: 0.31970450282096863
Batch_idx: 6   Loss: 0.19995659589767456
Batch_idx: 7   Loss: 0.1720093935728073
Batch_idx: 8   Loss: 0.16655562818050385
Batch_idx: 9   Loss: 0.1587594747543335
Batch_idx: 10   Loss: 0.32486799359321594
Batch_idx: 11   Loss: 0.11805446445941925
Batch_idx: 12   Loss: 0.12177381664514542
Batch_idx: 13   Loss: 0.15568186342716217
Batch_idx: 14   Loss: 0.2006593644618988
Batch_idx: 15   Loss: 0.2631950080394745
Batch_idx: 16   Loss: 0.20328184962272644
Batch_idx: 17   Loss: 0.20013852417469025
Batch_idx: 18   Loss: 0.13915075361728668
Batch_idx: 19   Loss: 0.2807193100452423
Batch_idx: 20   Loss: 0.19701215624809265
Batch_idx: 21   Loss: 0.30892518162727356
Batch_idx: 22   Loss: 0.22163912653923035
Batch_idx: 23   Loss: 0.129522

0.1980950214210294

## My network 

In [0]:
import torch.nn as nn 
# My Model 

class CNNModel(torch.nn.Module): 
  def __init__(self): 
    super(CNNModel, self).__init__() 

    self.conv_net = nn.Sequential(
        # Conv block 1 
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2), 
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),

        # Conv block 2 
        nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, stride=1, padding=2), 
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),

        # Conv block 3 
        nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding=1), 
        nn.ReLU(inplace=True),

        # Conv block 4 
        nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),

        # Conv block 5 
        nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )

    self.fc = nn.Sequential( 
        nn.Dropout(p=0.5), 
        nn.Linear(in_features=13824, out_features=4096, bias=True), 
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5), 
        nn.Linear(in_features=4096, out_features=4096, bias=True), 
        nn.ReLU(inplace=True), 
        nn.Linear(in_features=4096, out_features=1, bias=True)
    )


  def forward(self,x): 
    x = self.conv_net(x)
    x = x.view(x.size(0),-1)
    x = self.fc(x)
    return x 


In [0]:
myModel = CNNModel()
train(args,myModel,train_loader)

Train Epoch: 1 [16/33808 (0%)]	Loss: 0.256568
Train Epoch: 1 [336/33808 (1%)]	Loss: 0.128553
Train Epoch: 1 [656/33808 (2%)]	Loss: 0.275326
Train Epoch: 1 [976/33808 (3%)]	Loss: 0.076100
Train Epoch: 1 [1296/33808 (4%)]	Loss: 0.199798
Train Epoch: 1 [1616/33808 (5%)]	Loss: 0.358178
Train Epoch: 1 [1936/33808 (6%)]	Loss: 0.511225
Train Epoch: 1 [2256/33808 (7%)]	Loss: 0.204554
Train Epoch: 1 [2576/33808 (8%)]	Loss: 0.314032
Train Epoch: 1 [2896/33808 (9%)]	Loss: 0.379963
Train Epoch: 1 [3216/33808 (9%)]	Loss: 0.290904
Train Epoch: 1 [3536/33808 (10%)]	Loss: 0.161201
Train Epoch: 1 [3856/33808 (11%)]	Loss: 0.155005
Train Epoch: 1 [4176/33808 (12%)]	Loss: 0.199217
Train Epoch: 1 [4496/33808 (13%)]	Loss: 0.206063
Train Epoch: 1 [4816/33808 (14%)]	Loss: 0.269928
Train Epoch: 1 [5136/33808 (15%)]	Loss: 0.220694
Train Epoch: 1 [5456/33808 (16%)]	Loss: 0.248503
Train Epoch: 1 [5776/33808 (17%)]	Loss: 0.188702
Train Epoch: 1 [6096/33808 (18%)]	Loss: 0.202316
Train Epoch: 1 [6416/33808 (19%)]	Lo

In [0]:
test(myModel, test_loader)